In [1]:
import torch
import numpy as np
from tqdm import *
import os
import matplotlib.pyplot as plt
import PIL.Image

In [ ]:
def save_png_files_by_name(file_name, source_dir, target_dir,store_name):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    source_path = os.path.join(source_dir, file_name)
    with PIL.Image.open(source_path) as img:
        target_path = os.path.join(target_dir, store_name)
        img.save(target_path)

In [ ]:
def cal_metrics(TP,FP,FN):
    text_pre = TP/(TP+FP)
    text_recall = TP/(TP+FN)
    F_score = 2*text_pre*text_recall/(text_recall+text_pre)
    text_accu = TP/(TP+FP+FN)
    print(text_pre,text_recall,F_score,text_accu)

In [ ]:
sigma_cpu = torch.load("guidance_nv_track_trial2/sigma_28.pth")

"A city skyline at sunset with clouds forming the words 'Together we rise, apart we fall. Embrace unity!'"

In [5]:
prompt_idx = 2

In [ ]:
guidance_scale = 6
nv_gap_origion = torch.load("guidance_nv_track_trial2/prompt{}/dev-guidance{}/dev-guidance{}.pth".format(prompt_idx,guidance_scale,guidance_scale))

nv_gap_norm = torch.norm(nv_gap_origion,dim=[2,3])
nv_gap_norm = nv_gap_norm.permute(1,0)
noise_gap = nv_gap_norm*sigma_cpu

In [ ]:
total_classifiy = nv_gap_norm.permute(1,0)
classify_mean = torch.mean(total_classifiy,dim=-1)
sort_noise,sort_idx = torch.sort(classify_mean,descending=True)

source_dir = "guidance_nv_track_trial2/prompt{}/dev-guidance{}".format(prompt_idx,guidance_scale)
target_dir = "guidance_nv_track_trial2/prompt{}/sorted-guidance{}/total_sort".format(prompt_idx,guidance_scale)
for i in range(100):
    image_idx = sort_idx[i].item()
    cur_score = sort_noise[i].item()
    file_name = "dev-guidance{}-{}.png".format(guidance_scale,image_idx)
    target_name = "{}-{}-{}.png".format(i,cur_score,image_idx)
    save_png_files_by_name(file_name, source_dir, target_dir,target_name)

In [ ]:
time_step = 5
source_dir = "guidance_nv_track_trial2/prompt{}/dev-guidance{}".format(prompt_idx,guidance_scale)
best_k = 20

for i in range(5):
    target_dir = "guidance_nv_track_trial2/prompt{}/sorted-guidance{}/time_step_{}/best_{}".format(prompt_idx,guidance_scale,time_step,best_k)
    cur_nv = total_classifiy[:,:time_step]
    cur_mean = torch.mean(cur_nv,dim=-1)
    cur_sort,cur_idx = torch.sort(cur_mean,descending=True)
    for k in range(best_k):
        image_idx = cur_idx[k].item()
        cur_score = classify_mean[image_idx].item()
        file_name = "dev-guidance{}-{}.png".format(guidance_scale,image_idx)
        target_name = "{}-{}-{}.png".format(k,cur_score,image_idx)
        save_png_files_by_name(file_name, source_dir, target_dir,target_name)
    time_step+=5